In [32]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertForSequenceClassification,BertModel
from imblearn.over_sampling import SMOTE
from utils import CLSdata
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



In [35]:
from utils import CLSdata,SimpleCLSdata

csv_file = 'src/transformed/data.csv'
npy_dir = 'src/embedded_ef/'

# dataset = CLSdata(csv_file=csv_file, npy_dir=npy_dir)
dataset = SimpleCLSdata(csv_file=csv_file, npy_dir=npy_dir)
dataloader = DataLoader(dataset, batch_size=32, num_workers=0)

# Fetch the first item
# errorlist = 0
# for i in range(0,100000,5):
#     try:
#         dataset.__getitem__(i)
#     except:
#         errorlist +=1
# print(f"{errorlist}/{100000/5}")
# embed, label = dataset.__getitem__(1000)
# print(f"Embedding shape: {embed.shape}, Label: {label}")

ImportError: cannot import name 'SimpleCLSdata' from 'utils' (c:\Users\Jayden\Documents\school\capstone\utils.py)

In [33]:
npy_file = 'src/embedded_ef/batch_208.npy'
data = np.load(npy_file, mmap_mode='r')
print(data.shape)

(7892, 768)


In [12]:
for item in dataloader:
    print(item[0].shape)
    break

IndexError: Caught IndexError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "c:\Users\Jayden\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\_utils\worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Jayden\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Jayden\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "c:\Users\Jayden\Documents\school\capstone\utils.py", line 46, in __getitem__
    self.curr = np.load(self.npy_files[filereq])
                        ~~~~~~~~~~~~~~^^^^^^^^^
Inde

In [ ]:
dataset.labels

0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
          ... 
2367603    1.0
2367604    1.0
2367605    1.0
2367606    1.0
2367607    1.0
Name: troll, Length: 2367608, dtype: float64

**We Must split the data first**

In [4]:
batch_size = 2367608 // 8
batch_size

295951

In [ ]:
bert_model = BertModel.from_pretrained('bert-base-cased')


class BertClassifier(nn.Module):
    def __init__(self, bert_model):
        super(BertClassifier, self).__init__()
        self.bert = bert_model
        self.linear = nn.Linear(bert_model.config.hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask=None):
        print("Forward Pass")
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_hidden_state = outputs.last_hidden_state[:, 0, :]  # Using the hidden state corresponding to [CLS] token
        linear_output = self.linear(cls_hidden_state)
        output = self.sigmoid(linear_output)
        return output
    
model = BertClassifier(bert_model)

In [ ]:
num_epochs = 5  # Example number of epochs
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.BCELoss()

for epoch in range(num_epochs):
    print(f"------ Epoch {epoch} ------")
    for idx,batch in enumerate(dataloader):
        print("Batch",idx)
        optimizer.zero_grad()
        
        inputs, labels = batch
        
        # Tokenize input text
        # inputs_dict = tokenizer(inputs, return_tensors='pt', padding=True, truncation=True)
        print(inputs.shape)  
        # Forward pass
        outputs = model(inputs)
        
        # Calculate loss
        loss = criterion(outputs.squeeze(), labels.float())  # Assuming labels are 0/1
        print(f"Loss:{loss}")
        
        # Backward pass
        loss.backward()
        
        # Update weights
        optimizer.step()

------ Epoch 0 ------
Batch 0
torch.Size([8, 295951, 768])
Forward Pass


ValueError: too many values to unpack (expected 2)

In [ ]:
smote = SMOTE(sampling_strategy='auto')